<a href="https://colab.research.google.com/github/androvisck/CryptoRatioPanel/blob/main/Projeto_ratio_v05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**bold text**
*   symbol	= BTCUSDT, BNBUSDC, ETHBUSD...
*   period	= "5m","15m","30m","1h","2h","4h","6h","12h","1d"
*   limit	 = 30 ~~ 500
*   startTime
*   endTime

In [1]:
# instalando o python-binace e o vector
!pip install python-binance vectorbt

url = 'https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files'
!wget $url/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget $url/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib

     |████████████████████████████████| 65 kB 2.4 MB/s 
     |████████████████████████████████| 479 kB 17.0 MB/s 
     |████████████████████████████████| 288 kB 55.3 MB/s 
     |████████████████████████████████| 1.1 MB 40.7 MB/s 
     |████████████████████████████████| 111 kB 49.6 MB/s 
     |████████████████████████████████| 45 kB 3.2 MB/s 
     |████████████████████████████████| 3.4 MB 40.7 MB/s 
     |████████████████████████████████| 25.3 MB 1.4 MB/s 
     |████████████████████████████████| 94 kB 1.3 MB/s 
     |████████████████████████████████| 271 kB 13.7 MB/s 
     |████████████████████████████████| 144 kB 41.4 MB/s 
  Created wheel for vectorbt: filename=vectorbt-0.23.3-py3-none-any.whl size=526949 sha256=c40a8459585f18724b85a7f50ebcdb69f044c8d6394ae7f1ac73d3c70f6bfcd0
  Stored in directory: /root/.cache/pip/wheels/7b/0a/39/aa389a7b6ba6fe49e28081bd67cea4b870d429002b15dfa521
Successfully built vectorbt
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.

In [2]:
# seção de importação das bibliotecas
import nest_asyncio
nest_asyncio.apply()

import os
import sys
import datetime
from datetime import date, datetime, timezone, timedelta
import requests # Requests HTTP Library
import json 
import numpy
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sbn
from babel.numbers import format_decimal
import gc
import vectorbt as vbt
import talib as ta
from pandas.core.arrays import string_
import seaborn as sns

# Functions

In [3]:
def getAssets():
  link = 'https://fapi.binance.com/fapi/v1/exchangeInfo'
  temp = json.loads(requests.get(link).text)['symbols']
  asset=[]
  for i in temp:
    asset.append(i['symbol'])

  unwanted = {'DEFIUSDT','1000SHIBUSDT','BTCDOMUSDT','1000XECUSDT','1000BTTCUSDT'}
  asset = [j for j in asset if j not in unwanted]
  asset = [i for i in asset if not i.startswith('BTCUSDT_')]
  asset = [i for i in asset if not i.startswith('ETHUSDT_')]
  return asset

# getAssets()

In [4]:
def getLSR(symbol,timeframe,limite):
  if type(symbol) == str or type(timeframe) == str or type(limite) == str:
    link = 'https://fapi.binance.com/futures/data/globalLongShortAccountRatio?symbol='
    cont = symbol +'&period='+ timeframe +'&limit='+limite
    temp = json.loads(requests.get(link+cont).text) # carrega todo o resultado em um json
    data1 = []
    data = []
    for i in temp:
      data.append(i['timestamp'])

    for j in temp:  
      data1.append(j['longShortRatio'])

    num_float=[]
    for values in data1:
      num_float.append(float(values))
    data1 = [ round(elem, 4) for elem in num_float]
    return data,data1

# getLSR('BTCUSDT','5m','30')

In [5]:
def getVol(symbol,timeframe,limite):
  if type(symbol) == str or type(timeframe) == str or type(limite) == str:
    link = 'https://api.binance.com/api/v3/klines?symbol='
    cont = symbol +'&interval='+ timeframe +'&limit='+limite
    temp = json.loads(requests.get(link+cont).text) # carrega todo o resultado em um json

    # Volume de compra do Ativo
    data = []
    for i in temp:
      data.append(i[7])

    vol=[]
    for values in data:
        vol.append(round(float(values),6))
    data = [ round(elem, 6) for elem in vol]
    return data

# getVol('BTCUSDT','5m','30')

In [6]:
def getOI(symbol,timeframe,limite):
  if type(symbol) == str or type(timeframe) == str or type(limite) == str:
    link = 'https://fapi.binance.com/futures/data/openInterestHist?symbol='
    cont = symbol +'&period='+ timeframe +'&limit='+limite
    temp = json.loads(requests.get(link+cont).text) # carrega todo o resultado em um json

    # Volume de compra do Ativo
    data = []
    for i in temp:
      data.append(i['sumOpenInterest'])

    vol=[]
    for values in data:
        vol.append(float(values))
    data = [ round(elem, 6) for elem in vol]   
    return data

# getOI('BTCUSDT','5m','30')

In [7]:
def getData(symbol,timeframe,limite):
  for c in timeframe:
    if c.isalpha():
      if c == 'm':
        fator = int(timeframe.replace('m',''))
        unit = 'm'
      elif c == 'h':
        fator = int(timeframe.replace('h',''))
        unit = 'h'
      elif c == 'd':
        fator = int(timeframe.replace('d',''))
        unit = 'd'
  mult = str(limite*fator)
  nameClose = 'Cls' + str(timeframe)
  nameTrade = 'Trd' + str(timeframe)
  percentClose = 'Cls%' + str(timeframe)
  rsiClose  = 'Cls' + str(timeframe) + 'RSI'
  rsiTrade  = 'Trd' + str(timeframe) + 'RSI'
  temp = vbt.BinanceData.download(symbol, start=mult+unit, interval=timeframe).get()
  data = temp[['Close','Number of trades']].iloc[[len(temp)-1]]
  data.columns = [nameClose,nameTrade]
  data[rsiClose]=ta.RSI(temp['Close'],timeperiod=14)
  data[rsiTrade]=ta.RSI(temp['Number of trades'],timeperiod=14)
  data[percentClose]=((temp['Close'][-1]-temp['Close'][-2])/temp['Close'][-1])
  data = data.reset_index(drop=True)
  return data

# getData('BTCUSDT','5m',30)

In [8]:
# assets e configurações prévias
#--------------------------------------------------------
period = ["1m","5m","15m","1h","4h","1d"]
limite = '30'
symbol = getAssets()
# df = pd.DataFrame()
#---------------------------------------------------------
# period = ["5m"]
# symbol = ['BTCUSDT', 'ETHUSDT', 'ONEUSDT', 'DOTUSDT', 'CHZUSDT']

In [9]:
passou = False
while passou == False:
  # asset=pd.DataFrame()
  try:
    del [asset]
  except:
    for z in symbol:      
      for y in ['5m']:

        lsr = getLSR(z,y,limite)
        oi = getOI(z,y,limite)    
        vol = getVol(z,y,limite) 
        # print(len(lsr[0]),len(oi),len(vol))
        # print('Passou!')

        try:
          del [df]
          gc.collect()
          df=pd.DataFrame()
        except:
          print("Something else went wrong")
        df = pd.DataFrame(
            {'time': lsr[0][-15:],
            'symbol': z,
            'lsr': lsr[1][-15:],
            'lsrRSI': ta.RSI(numpy.asarray(lsr[1][-30:]),timeperiod=14)[-15:],
            'vol': vol[-15:],
            'volRSI': ta.RSI(numpy.asarray(vol[-30:]),timeperiod=14)[-15:],
            'oi': numpy.array(oi[-15:],dtype='f8'),
            'oiRSI': ta.RSI(numpy.array(oi[-30:],dtype='f8'),timeperiod=14)[-15:],
            })
      
        day_divider = 86400000
        x = df['time'].values.astype(dtype='datetime64[ms]') # for msec format
        x = numpy.array(x)

        df['time'] = pd.to_datetime(x)
        df.index = pd.to_datetime(df['time'], unit='s')
        df = df.drop(columns=['time'])

        # print(z + " / " + y)
        try:
          if not asset.empty:
            asset = asset.append(df.iloc[-1:], ignore_index = True)
        except:
          asset = df.iloc[-1:]

  passou = True

Something else went wrong


In [10]:
# asset

In [ ]:
try:
  if not asset.empty:
    for i in symbol:
      try:
        del [data, temp]
      except:
        pass
      for j in period:
        temp = getData(i,j,30)
        try:
          if not data.empty:
            data = pd.concat([data,temp], axis=1)
        except:
          data = temp
      try:
        if not info.empty:
          info = info.append(data.iloc[-1:], ignore_index = True)
      except:
        info = data.iloc[-1:]

    # info = pd.concat([asset,info], axis=1)
except:
  print("Asset Vazio!!!")

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [ ]:
info = pd.concat([asset,info], axis=1)
infomatio = info.loc[:,~info.columns.duplicated()]
panel = infomatio[['symbol','Cls1m','Cls%1h','Cls%4h','Cls%1d','Trd1h','Trd4h','Trd1d','lsr','Cls1mRSI','Cls5mRSI','Cls15mRSI','Cls1hRSI','Cls4hRSI','Cls1dRSI','Trd1mRSI','volRSI','lsrRSI','oiRSI']]

In [ ]:
df

In [ ]:
# infomatio

In [ ]:
# panel

In [ ]:
try:
  infomatio['oi'] = infomatio['oi'].astype(int)
  infomatio['oi'] = (infomatio['oi']/1000).apply(lambda x: '${:,.2f}k'.format(x))
except:
  pass

In [ ]:
def background_with_norm(s):
    cmap = matplotlib.cm.get_cmap('RdYlGn')
    min = numpy.min(s)
    max = numpy.max(s)
    if min < 0 and max < 0:
      max = 1
      norm = matplotlib.colors.DivergingNorm(vmin=min, vcenter=0, vmax=max)
    elif min > 0 and max > 0:
      min = -1
      norm = matplotlib.colors.DivergingNorm(vmin=min, vcenter=0, vmax=max)
    else:
      norm = matplotlib.colors.DivergingNorm(vmin=min, vcenter=0, vmax=max)

    return ['background-color: {:s}'.format(matplotlib.colors.to_hex(c.flatten())) for c in cmap(norm(s.values))]

all_ = ['Cls1m','Cls%1h','Cls%4h','Cls%1d','Trd1h','Trd4h','Trd1d','lsr','Cls1mRSI','Cls5mRSI','Cls15mRSI','Cls1hRSI','Cls4hRSI','Cls1dRSI','Trd1mRSI','volRSI','lsrRSI','oiRSI']
closes = ['Cls%1h','Cls%4h','Cls%1d']
trades = ['Trd1h', 'Trd4h', 'Trd1d']
rsi = ['Cls1mRSI',	'Cls5mRSI',	'Cls15mRSI',	'Cls1hRSI',	'Cls4hRSI',	'Cls1dRSI','Trd1mRSI']
# strings = ['Cls1m','Cls%1h','Cls%4h','Cls%1d','oi','Trd1h','Trd4h',
#            'Trd1d','lsr','Cls1mRSI','Cls5mRSI','Cls15mRSI','Cls1hRSI','Cls4hRSI','Cls1dRSI','Trd1mRSI','volRSI','lsrRSI','oiRSI']

In [ ]:
panel[:int(len(asset))].style.hide_index().bar(subset=trades, vmin=0, color='#0143da').set_properties(**{'background-color': 'black','color': 'white','border-color': 'white','text-align': 'center'})\
.background_gradient('Spectral_r', subset='lsr').apply(background_with_norm,subset=closes).text_gradient(cmap='RdYlGn_r', subset=rsi)\
.format('{:.2f}', subset=all_).format('{:.4f}', subset='Cls1m').format('{:.2%}', subset=closes).set_properties(subset=closes,color="#00013c").set_properties(subset=trades,background_color='white')\
.text_gradient(cmap="RdYlGn_r", vmin=30, vmax=70, subset= ['volRSI','oiRSI','lsrRSI'])

In [ ]:
x = df['lsr']
y = df['oi']
# y = y.iloc[::-1]
plt.scatter(x, y) 
plt.plot(numpy.unique(x), numpy.poly1d(numpy.polyfit(x, y, 1))(numpy.unique(x)), color='red') 

In [ ]:
x = df['lsrRSI']
y = df['volRSI']
plt.scatter(x, y) 
plt.plot(numpy.unique(x), numpy.poly1d(numpy.polyfit(x, y, 1))(numpy.unique(x)), color='red') 

In [ ]:
x = df['volRSI']
y = df['oiRSI']
plt.scatter(x, y) 
plt.plot(numpy.unique(x), numpy.poly1d(numpy.polyfit(x, y, 1))(numpy.unique(x)), color='red') 

In [ ]:
import seaborn as sbn
from functools import reduce
dfs = [df['lsr'],df['lsrRSI'],df['vol'],df['volRSI'],df['oi'],df['oiRSI']]
# dfs = [x,y]
final = reduce(lambda left,right: pd.merge(left,right,on='time',how='inner'),dfs)
plt.figure(figsize=(16,9))
sbn.heatmap(final.corr(),vmin=0, vmax=1,annot=True, cmap='BrBG')

In [ ]:
df.head()

In [ ]:
url = 'https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files'
!wget $url/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget $url/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib
import talib

In [ ]:
import talib as ta

n = 300

a = df_ativo.index[-int(n):]
b = df_ativo['close'][-int(n):]
c = df.index[-n:]
d = df["ratio"][-n:]

rateRSI = ta.RSI(d, timeperiod=14)
rateRSI = ta.BETA(d, rateRSI, timeperiod=7)

plt.close('all')
fig, ax1 = plt.subplots(1, 1, figsize=(16, 6))

# Asset
ax3 = ax1.twinx()
ax3.plot(c, b, linewidth=1.5, markersize=15, color='red')
ax3.tick_params(axis="y", labelcolor='red')
ax3.set_title("Price VS Interest VS Ratio ["+symb+"]", color='black', fontsize=24)
# ax3.get_yaxis().set_visible(False)
ax3.spines['right'].set_position('center')



# Ratio
ax2 = ax1.twinx()
ax2.plot(c, d, linewidth=2, markersize=15, color='blue')
ax2.set_ylabel('lsr', color='blue', fontsize=24)
ax2.tick_params(axis="y", labelcolor='blue')
minr = numpy.min(df['lsr'])
maxr = numpy.max(df['lsr'])
# ax2.set_ylim([minr - (minr / 100), maxr + (maxr / 100)])
# ax2.set_xlim(minr,maxr)
sbn.set_style("whitegrid")


# RSI_rate
ax1.plot(c, rateRSI, linewidth=1.5, markersize=15, color='black')
ax1.set_ylabel('RSI Rate', color='black', fontsize=24)
ax1.tick_params(axis="y", labelcolor='black')
ax1.tick_params(axis='x', rotation=70)
ax1.ticklabel_format(axis="y", style='plain')
mina = numpy.min(rateRSI)
maxa = numpy.max(rateRSI)
ax1.set_ylim([mina , maxa])
# x_ticks = numpy.linspace(ax1.get_xticks()[0], ax1.get_xticks()[-1], l)
# ax1.set_yticks(x_ticks)
sbn.set_style("whitegrid")


# Determine which plot has finer grid. Set pointers accordingly
l1 = len(ax1.get_yticks())
l2 = len(ax2.get_yticks())
if l1 > l2:
    a = ax1
    b = ax2
    c = ax3
    l = l1
else:
    a = ax2
    b = ax1
    c = ax3
    l = l2
# Respace grid of 'b' axis to match 'a' axis
c_ticks = numpy.linspace(c.get_yticks()[0], c.get_yticks()[-1], l)
c.set_yticks(c_ticks)
b_ticks = numpy.linspace(b.get_yticks()[0], b.get_yticks()[-1], l)
b.set_yticks(b_ticks)
a_ticks = numpy.linspace(a.get_yticks()[0], a.get_yticks()[-1], l)
a.set_yticks(a_ticks)
ax2.grid(None)
fig.tight_layout()

In [ ]:
# !pip install -U "vectorbt[full]"
!pip install vectorbt

# Order Book


In [ ]:
symbol = 'MTLUSDT'
symb = symbol[:-4]
limit = '30'
period = '5m'
lim = '30'

In [ ]:
limit = 5000 # Default 100. Valid limits:[5, 10, 20, 50, 100, 500, 1000, 5000]
res = requests.get('https://api.binance.com/api/v3/depth?symbol=' + symbol + '&limit=' + str(limit)) # to spot book

data = json.loads(res.text) # carrega todo o resultado em um json
x = [[k,v]for k,v in data.items()]# x é uma lista de listas

bid = pd.DataFrame(x[1][1], columns=['price','qtty']) # dataframes to spot book
ask = pd.DataFrame(x[2][1], columns=['price','qtty']) # dataframes to spot book
bid['price'] = pd.to_numeric(bid['price'], errors='coerce')
bid['qtty'] = pd.to_numeric(bid['qtty'], errors='coerce')
ask['price'] = pd.to_numeric(ask['price'], errors='coerce')
ask['qtty'] = pd.to_numeric(ask['qtty'], errors='coerce')

t = bid.sort_values('qtty')[-26:] # 15 maiores
u = ask.sort_values('qtty')[-25:] # 15 maiores

In [ ]:
t = t[:-1]

In [ ]:
conB = u.sort_values('price')
conB = conB[1:]
conA = t.sort_values('price')

conB = conB.groupby(by=conB.price, as_index=False).sum()
conA = conA.groupby(by=conA.price, as_index=False).sum()

conB = conB.sort_values('qtty')
conA = conA.sort_values('qtty')

conB['id'] = numpy.arange(0,len(conB['qtty']))
conA['id'] = numpy.arange(0,len(conA['qtty']))

conB = conB.sort_values('price')
conA = conA.sort_values('price')

rankB = conB['id'].tolist()
rankA = conA['id'].tolist()

sigmA =[]
sigmB =[]
gr = sbn.color_palette("YlGn", len(rankA))
rd = sbn.color_palette("OrRd", len(rankB))
for i in rankA:
  sigmA.append(gr[i])
for j in rankB:
  sigmB.append(rd[j])

sigm = sigmA+sigmB

con = pd.concat([conB,conA])

kwargs = {'alpha':0.9, 'linewidth':0.5, 'edgecolor':'black'}
ax = sbn.barplot(x="price", y="qtty", data=con, **kwargs)
ax.tick_params(axis='x', rotation=70)
ax.set_title(symb + ' [Futures Book  Orders]', fontsize=24)

for i in range(1, len(ax.patches)+1):
  if i < round(len(ax.patches)/2):
    bar = ax.patches[-i]
    for j in range(len(conB['qtty'])):
      if bar.get_height() == conB['qtty'][j]:
        bar.set_color(sigmB[j])
        bar.set_edgecolor('k')

  elif i > round(len(ax.patches)/2):
    bar = ax.patches[-i]
    for K in range(len(conA['qtty'])):
      if bar.get_height() == conA['qtty'][K]:
        bar.set_color(sigmA[K])
        bar.set_edgecolor('k')

ax.ticklabel_format(axis="y", style='plain') # non scientifical notation
ax.set_ylabel('Amount [USDT]', fontsize=24) # set ylable and fontsize
ax.set_xlabel('') # set ylable and fontsize

# ax.axhline(int(fg), ls='--',color ="green", label='BID (75%) > $'+ str(format_decimal(fh, locale='en_US')))
# ax.axhline(int(fh), ls='--', color ="red", label='ASK (75%) > $'+ str(format_decimal(fg, locale='en_US')))
# leg = plt.legend()
# plt.axvline(y=h, color='blue', ls='--', lw=2.5)
sbn.set(rc={"figure.figsize":(18, 5)})
sbn.set_style("whitegrid")

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15,4), sharey='row')
fig.tight_layout()
ax[0].hist(bid.price, bins=20, ec='black', color='green')
ax[0].set_xlim([t.price.min(),t.price.max()])
ax[1].hist(ask.price, bins=20, ec='black', color='red')
ax[0].set_title('[Bid Orders]', fontsize=24)
ax[1].set_title('[Ask Orders]', fontsize=24)
ax[0].set_ylabel('Amount [USDT]', fontsize=24) # set ylable and fontsize
ax[1].set_xlim([u.price.min(),u.price.max()])
plt.show()

# Futures Book

In [ ]:
limit = 1000 # 5, 10, 20, 50, 100, 500, 1000
fres = requests.get('https://fapi.binance.com/fapi/v1/depth?symbol=' + symbol + '&limit=' + str(limit)) # to spot book

fdata = json.loads(fres.text) # carrega todo o resultado em um json
fx = [[k,v]for k,v in fdata.items()]# x é uma lista de listas

fbid = pd.DataFrame(fx[4][1], columns=['price','qtty']) # dataframes to futures book
fask = pd.DataFrame(fx[3][1], columns=['price','qtty']) # dataframes to futures book

fbid['price'] = pd.to_numeric(fbid['price'], errors='coerce')
fbid['qtty'] = pd.to_numeric(fbid['qtty'], errors='coerce')
fask['price'] = pd.to_numeric(fask['price'], errors='coerce')
fask['qtty'] = pd.to_numeric(fask['qtty'], errors='coerce')
# bid.sort_values('qtty')[-13:]

ft = fbid.sort_values('qtty').round(4)[-25:] # 25 maiores
fu = fask.sort_values('qtty').round(4)[-25:] # 25 maiores

In [ ]:
conB = ft.sort_values('price')
conA = fu.sort_values('price')

conB = conB.groupby(by=conB.price, as_index=False).sum()
conA = conA.groupby(by=conA.price, as_index=False).sum()

conB = conB.sort_values('qtty')
conA = conA.sort_values('qtty')

conB['id'] = numpy.arange(0,len(conB['qtty']))
conA['id'] = numpy.arange(0,len(conA['qtty']))

conB = conB.sort_values('price')
conA = conA.sort_values('price')

rankB = conB['id'].tolist()
rankA = conA['id'].tolist()

sigmA =[]
sigmB =[]
gr = sbn.color_palette("YlGn", len(rankA))
rd = sbn.color_palette("autumn_r", len(rankB))
for i in rankA:
  sigmA.append(gr[i])
for j in rankB:
  sigmB.append(rd[j])

sigm = sigmA+sigmB

con = pd.concat([conB,conA])

# Keyword Arguments parameter
kwargs = {'alpha':0.9, 'linewidth':0.5, 'edgecolor':'black'}
ax = sbn.barplot(x="price", y="qtty", data=con, **kwargs)
ax.tick_params(axis='x', rotation=70)
ax.set_title(symb + ' [Futures Book  Orders]', fontsize=24)

for i in range(1, len(ax.patches)+1):
  if i < round(len(ax.patches)/2):
    bar = ax.patches[-i]
    for j in range(len(conB['qtty'])):
      if bar.get_height() == conB['qtty'][j]:
        bar.set_color(sigmB[j])
        bar.set_edgecolor('k')

  elif i > round(len(ax.patches)/2):
    bar = ax.patches[-i]
    for K in range(len(conA['qtty'])):
      if bar.get_height() == conA['qtty'][K]:
        bar.set_color(sigmA[K])
        bar.set_edgecolor('k')

ax.ticklabel_format(axis="y", style='plain') # non scientifical notation
ax.set_ylabel('Amount [USDT]', fontsize=24) # set ylable and fontsize
ax.set_xlabel('') # set ylable and fontsize

# ax.axhline(int(fg), ls='--',color ="green", label='BID (75%) > $'+ str(format_decimal(fh, locale='en_US')))
# ax.axhline(int(fh), ls='--', color ="red", label='ASK (75%) > $'+ str(format_decimal(fg, locale='en_US')))
# leg = plt.legend()
# plt.axvline(y=h, color='blue', ls='--', lw=2.5)
sbn.set(rc={"figure.figsize":(18, 5)})
sbn.set_style("whitegrid")

In [ ]:
1
# asset=pd.DataFrame()
try:
  del [asset]
except:
  for z in symbol:      
    for y in period:
      if 'm' in y:
        x = y.split('m')[0]
        x = int(x)
        limit = x * int(lim)
      elif 'h' in y:
        x = y.split('h')[0]
        x = int(x)
        limit = x * int(lim) * 60
      elif 'd' in y:
        x = y.split('d')[0]
        x = int(x)
        limit = x * int(lim) * 60 * 24

      symb = symbol[:-4]

      resp = requests.get('https://fapi.binance.com/futures/data/globalLongShortAccountRatio?symbol='+
                          z +'&period='+ y +'&limit='+lim)
      json_data = json.loads(resp.text) # carrega todo o resultado em um json

      res = []
      for sub in json_data:
        res.append(sub['longShortRatio'])

      num_float=[]
      for values in res:
          num_float.append(float(values))
      res = [ round(elem, 4) for elem in num_float ]

      ind = []
      for sub in json_data:
        ind.append(sub['timestamp'])

      response_ativo = requests.get('https://api.binance.com/api/v3/klines?symbol='+
                                z +'&interval='+ y +'&limit='+lim)
      json_ativo = json.loads(response_ativo.text) # carrega todo o resultado em um json

      # Close price
      res_ativo4 = []
      for sub in json_ativo:
        res_ativo4.append(sub[4])
      # print(res_ativo4)

      # Volume de compra do Ativo
      res_ativo7 = []
      for sub in json_ativo:
        res_ativo7.append(sub[5])
      # print(res_ativo7)

      num_float4=[]
      for values in res_ativo4:
          num_float4.append(round(float(values),6))
      res_ativo4 = [ round(elem, 6) for elem in num_float4 ]

      num_float7=[]
      for values in res_ativo7:
          num_float7.append(round(float(values),6))
      res_ativo7 = [ round(elem, 6) for elem in num_float7 ]

      ind_ativo = []
      for sub in json_ativo:
        ind_ativo.append(sub[6])

      resp = requests.get('https://api.binance.com/api/v3/depth?symbol=' + z + '&limit=' + str(limit)) # to spot book
      data = json.loads(resp.text) # carrega todo o resultado em um json
      x = [[k,v]for k,v in data.items()]# x é uma lista de listas

      bid = pd.DataFrame(x[1][1], columns=['price','qtty']) # dataframes to spot book
      ask = pd.DataFrame(x[2][1], columns=['price','qtty']) # dataframes to spot book
      bid_price = pd.to_numeric(bid['price'], errors='coerce')[-30:]
      bid_qtty = pd.to_numeric(bid['qtty'], errors='coerce')[-30:]
      ask_price = pd.to_numeric(ask['price'], errors='coerce')[-30:]
      ask_qtty = pd.to_numeric(ask['qtty'], errors='coerce')[-30:]


      ans = requests.get('https://fapi.binance.com/futures/data/openInterestHist?symbol='+
                      z +'&period='+ y +'&limit='+lim)
      json_ans = json.loads(ans.text) # carrega todo o resultado em um json

      res_open = []
      for sub in json_ans:
        res_open.append(sub['sumOpenInterest'])

      num_float=[]
      for values in res:
          num_float.append(float(values))
      res = [ round(elem, 4) for elem in num_float ]

      ind = []
      for sub in json_ans:
        ind.append(sub['timestamp'])
      
      
      try:
        del [df]
        gc.collect()
        df=pd.DataFrame()
      except:
        print("Something else went wrong")


      df = pd.DataFrame(
          {'time': ind,
          'symbol': z,
          'lsr': res[-30:],
          'close': res_ativo4[-30:],
          # 'closeRSI': ta.RSI(numpy.asarray(res_ativo4),timeperiod=14)[-30:],
          'vol': res_ativo7[-30:],
          # 'volRSI': ta.RSI(numpy.asarray(res_ativo7),timeperiod=14)[-30:],
          'bid_qtt': bid_qtty[-30:],
          'bid_price': bid_price[-30:],
          'ask_qtt': numpy.array(ask_qtty[-30:],dtype='f8'),
          'ask_price': numpy.array(ask_price[-30:],dtype='f8'),
          # 'oi': res_open[-30:],
          # 'oiRSI': ta.RSI(numpy.array(res_open[-30:],dtype='f8'),timeperiod=14)[-30:],
          })
      
      day_divider = 86400000
      x = df['time'].values.astype(dtype='datetime64[ms]') # for msec format
      x = numpy.array(x)

      df['time'] = pd.to_datetime(x)
      df.index = pd.to_datetime(df['time'], unit='s')
      df = df.drop(columns=['time'])

      print(z + " / " + y)
      try:
        if not asset.empty:
          asset = asset.append(df.iloc[-1:], ignore_index = True)
      except:
        asset = df.iloc[-1:]

# Open Interest

In [ ]:
ans = requests.get('https://fapi.binance.com/futures/data/openInterestHist?symbol='+
                    symbol+'&period='+period+'&limit='+lim)
json_ans = json.loads(ans.text) # carrega todo o resultado em um json

res = []
for sub in json_ans:
  res.append(sub['sumOpenInterest'])

num_float=[]
for values in res:
    num_float.append(float(values))
res = [ round(elem, 4) for elem in num_float ]

ind = []
for sub in json_ans:
  ind.append(sub['timestamp'])

df_open = pd.DataFrame(
    {'time': ind,
     'interest': res
    })

In [ ]:
day_divider = 86400000
x = df_open['time'].values.astype(dtype='datetime64[ms]') # for msec format
x = numpy.array(x)

df_open['time'] = pd.to_datetime(x)
df_open.index = pd.to_datetime(df_open['time'], unit='s')
df_open = df_open.drop(columns=['time'])

min = numpy.min(df_open['interest'])
max = numpy.max(df_open['interest'])

In [ ]:
n = 30

a = df_ativo.index[-int(n):]
b = df_ativo['close'][-int(n):]
c = df.index[-n:]
d = df["ratio"][-n:]
e = df_open.index[-int(n):]
f = df_open['interest'][-int(n):]

plt.close('all')
fig, ax1 = plt.subplots(1, 1, figsize=(16, 6))

# Asset
ax3 = ax1.twinx()
ax3.plot(e, b, linewidth=1.5, markersize=15, color='red')
ax3.tick_params(axis="y", labelcolor='red')
ax3.set_title("Price VS Interest VS Ratio ["+symb+"]", color='black', fontsize=24)
# ax3.get_yaxis().set_visible(False)
ax3.spines['right'].set_position('center')



# Ratio
ax2 = ax1.twinx()
ax2.plot(e, d, linewidth=2, markersize=15, color='blue')
ax2.set_ylabel('lsr', color='blue', fontsize=24)
ax2.tick_params(axis="y", labelcolor='blue')
minr = numpy.min(df['lsr'])
maxr = numpy.max(df['lsr'])
# ax2.set_ylim([minr - (minr / 100), maxr + (maxr / 100)])
# ax2.set_xlim(minr,maxr)
sbn.set_style("whitegrid")


# Open Interest
ax1.plot(e, f, linewidth=1.5, markersize=15, color='black')
ax1.set_ylabel('Interest [USDT]', color='black', fontsize=24)
ax1.tick_params(axis="y", labelcolor='black')
ax1.tick_params(axis='x', rotation=70)
ax1.ticklabel_format(axis="y", style='plain')
mina = numpy.min(df_open['interest'])
maxa = numpy.max(df_open['interest'])
ax1.set_ylim([mina , maxa])
# x_ticks = numpy.linspace(ax1.get_xticks()[0], ax1.get_xticks()[-1], l)
# ax1.set_yticks(x_ticks)
sbn.set_style("whitegrid")


# Determine which plot has finer grid. Set pointers accordingly
l1 = len(ax1.get_yticks())
l2 = len(ax2.get_yticks())
if l1 > l2:
    a = ax1
    b = ax2
    c = ax3
    l = l1
else:
    a = ax2
    b = ax1
    c = ax3
    l = l2
# Respace grid of 'b' axis to match 'a' axis
c_ticks = numpy.linspace(c.get_yticks()[0], c.get_yticks()[-1], l)
c.set_yticks(c_ticks)
b_ticks = numpy.linspace(b.get_yticks()[0], b.get_yticks()[-1], l)
b.set_yticks(b_ticks)
a_ticks = numpy.linspace(a.get_yticks()[0], a.get_yticks()[-1], l)
a.set_yticks(a_ticks)
ax2.grid(None)
fig.tight_layout()

# TESTES

In [ ]:
conB = ft.sort_values('price')
conA = fu.sort_values('price')

conB = conB.groupby(by=conB.price, as_index=False).sum()
conA = conA.groupby(by=conA.price, as_index=False).sum()

conB = conB.sort_values('qtty')
conA = conA.sort_values('qtty')

conB['id'] = numpy.arange(0,len(conB['qtty']))
conA['id'] = numpy.arange(0,len(conA['qtty']))

conB = conB.sort_values('price')
conA = conA.sort_values('price')

rankB = conB['id'].tolist()
rankA = conA['id'].tolist()

sigmA =[]
sigmB =[]
gr = sbn.color_palette("YlGn", len(rankA))
rd = sbn.color_palette("autumn_r", len(rankB))
for i in rankA:
  sigmA.append(gr[i])
for j in rankB:
  sigmB.append(rd[j])

sigm = sigmA+sigmB

con = pd.concat([conB,conA])
# con = con.iloc[::-1]

# Keyword Arguments parameter
kwargs = {'alpha':0.9, 'linewidth':0.5, 'edgecolor':'black'}
ax = sbn.barplot(y="price", x="qtty", data=con, order=con.sort_values('price',ascending = False).price, orient='h', **kwargs)
ax.tick_params(axis='x', rotation=70)
ax.set_title(symb + ' [Futures Book  Orders]', fontsize=24)

sample = len(ax.patches)
for i in range(1, sample+1):
  if i < round(sample/2):
    bar = ax.patches[-i]
    for j in range(len(conB['qtty'])):
      width = "{:1.0f}".format(bar.get_width())
      if width == conB['qtty'][j]:
        bar.set_color(sigmA[j])
        bar.set_edgecolor('k')

    # for p in ax.patches[:int((sample-1)/2)]:
    #   width = int("{:1.0f}".format(p.get_width()))
    #   for j in range(len(conA['qtty'])):
    #     if width == conA['qtty'][j]:
    #       p.set_color(sigmA[j])
    #       p.set_edgecolor('k')

  elif i > round(sample/2):
    bar = ax.patches[-i]
    for p in ax.patches[int((sample-1)/2):]:
      width = int("{:1.0f}".format(p.get_width()))
      for K in range(len(conB['qtty'])):
        width = "{:1.0f}".format(bar.get_width())
        if width == conB['qtty'][K]:
          p.set_color(sigmB[K])
          p.set_edgecolor('k')

ax.ticklabel_format(axis="x", style='plain') # non scientifical notation
ax.set_ylabel('Price [USDT]', fontsize=24) # set ylable and fontsize
ax.set_xlabel('Amount [USDT]', fontsize=24) # set ylable and fontsize

sbn.set(rc={"figure.figsize":(5, 10)})
sbn.set_style("whitegrid")

In [ ]:
kwargs = {'alpha':0.9, 'linewidth':0.5, 'edgecolor':'black'}
ax = sbn.barplot(y="price", x="qtty", data=con, order=con.sort_values('price',ascending = False).price, orient='h', **kwargs)
ax.tick_params(axis='x', rotation=70)
ax.set_title(symb + ' [Futures Book  Orders]', fontsize=24)

initialx = 0
for p in ax.patches:
  ax.text(p.get_width(),initialx+p.get_height()/4,"{:1.0f}".format(p.get_width()))
  initialx += 1


ax.ticklabel_format(axis="x", style='plain') # non scientifical notation
ax.set_ylabel('Price [USDT]', fontsize=24) # set ylable and fontsize
ax.set_xlabel('Amount [USDT]', fontsize=24) # set ylable and fontsize

sbn.set(rc={"figure.figsize":(5, 20)})
sbn.set_style("whitegrid")


# WEBSOCKET

In [ ]:
# https://livedataframe.com/live-cryptocurrency-data-python-tutorial/

from binance.client import Client
# from binance.websockets import BinanceSocketManager 
from datetime import datetime
import time
# Although fine for tutorial purposes, your API Keys should never be placed directly in the script like below. 
# You should use a config file (cfg or yaml) to store them and reference when needed.
PUBLIC = '<YOUR-PUBLIC-KEY>'
SECRET = '<YOUR-SECRET-KEY>'

client = Client(api_key=PUBLIC, api_secret=SECRET)
bm = BinanceSocketManager(client)

def handle_message(msg):
    
    if msg['e'] == 'error':    
        print(msg['m'])
        
    else:
        # Bitcoins exchanged - This time converting the strings to floats.
        bitcoins_exchanged = float(msg['p']) * float(msg['q'])
        
        # Make time pretty
        timestamp = msg['T'] / 1000
        timestamp = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')
        
        # Buy or sell?
        if msg['m'] == True:
            event_side = 'SELL'
        else:
            event_side = 'BUY '
        
        # Print this amount
        print("{} - {} - {} - Price: {} - Qty: {} BTC Qty: {}".format(timestamp,
                                                                       event_side,
                                                                       msg['s'],
                                                                       msg['p'],
                                                                       msg['q'],
                                                                       bitcoins_exchanged))

conn_key = bm.start_trade_socket('ETHBTC', handle_message)

bm.start()

time.sleep(5) # let some data flow..

bm.stop_socket(conn_key)